In [1]:
import findspark
findspark.init()
import requests
import xml.etree.ElementTree as ET
from pyspark import SparkContext
from pyspark.sql.functions import * #desc, col
from pyspark.sql import *
from pyspark.sql.types import *

In [2]:
# Create session
spark = SparkSession.builder.getOrCreate()
spark

In [3]:
# Create a new data frame for movie_review.csv
log_df = spark.read.csv('C:/Users/maryf/Capstone project/log_reviews.csv', sep=',', header=True)

In [13]:
# Take a little part of the data frame to make tests.
mini_df = log_df.limit(100000)
#mini_df.printSchema()

In [14]:
# Create a schema to keep the data that comes from xml
extract_log_info_schema = StructType([
    StructField("logDate", StringType(), True),
    StructField("device", StringType(), True),
    StructField("location", StringType(), True),
    StructField("os", StringType(), True),
    StructField("ipAddress", StringType(), True),
    StructField("phoneNumber", StringType(), True),
])

# Define a function to obtain the selected data from xml
def select_text(doc, xpath):
    nodes = [e.text for e in doc.findall('./log/'+ xpath) if isinstance(e, ET.Element)]
    return next(iter(nodes), None)

# Define a function to extract the data
def extract_log_info(payload):
    doc = ET.fromstring(payload)
    return {
        'logDate':  select_text(doc, 'logDate'),
        'device': select_text(doc, 'device'),
        'location': select_text(doc, 'location'),
        'os': select_text(doc, 'os'),
        'ipAddress': select_text(doc, 'ipAddress'),
        'phoneNumber': select_text(doc, 'phoneNumber')
        }
# Create a udf function
extract_log_info_udf = udf(extract_log_info, extract_log_info_schema)

In [15]:
another_df = mini_df.withColumn("info", extract_log_info_udf('log')).select('id_review', 'info.logDate', 'info.device', 'info.location', 'info.os', 'info.ipAddress', 'info.phoneNumber')
another_df.show()

+---------+----------+--------+-------------+-----------------+------------+------------+
|id_review|   logDate|  device|     location|               os|   ipAddress| phoneNumber|
+---------+----------+--------+-------------+-----------------+------------+------------+
|        1|04-25-2021|  Mobile|       Kansas|        Apple iOS|9.200.232.57|821-540-5777|
|        2|03-13-2021|  Tablet|       Oregon|   Google Android|9.200.232.57|819-102-1320|
|        3|09-30-2021|  Tablet|    Minnesota|        Apple iOS|9.200.232.57|989-156-0498|
|        4|05-24-2021|  Tablet|     Arkansas|      Apple MacOS|9.200.232.57|225-837-9935|
|        5|02-01-2021|  Tablet|New Hampshire|            Linux|9.200.232.57|243-842-4562|
|        6|07-23-2021|  Tablet|  Pensylvania|        Apple iOS|9.200.232.57|694-501-4352|
|        7|10-13-2021|Computer|     New York|      Apple MacOS|9.200.232.57|430-449-7136|
|        8|09-18-2021|Computer|   California|Microsoft Windows|9.200.232.57|633-661-7714|
|        9